#Ms. Pacman Agent
##Policy Gradient with Neural Network
References:
- _Hugging Face policy gradient_ https://huggingface.co/learn/deep-rl-course/unit4/hands-on

In [ ]:
!pip install gymnasium[atari]
!pip install gymnasium[accept-rom-license]
!pip install tensorflow
!pip install numpy
!pip install torch
!pip install imageio[ffmpeg]

In [ ]:
# Virtual display
# Only needed for Google Colab
# Code from CS 175 HW 2
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip install pyvirtualdisplay
!pip install pyglet==1.5.1

from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [3]:
import os
import gymnasium as gym
from collections import deque

import numpy as np
import matplotlib.pyplot as plt
from gymnasium import envs
from tqdm import tqdm

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

import imageio

env = gym.make("ALE/MsPacman-ram-v5")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(torch.cuda.is_available())

In [ ]:
# print("_____OBSERVATION SPACE_____ \n")
# print("Observation Space", env.observation_space)
# print("Sample observation", env.observation_space.sample()) # Get a random observation

In [ ]:
# print("\n _____ACTION SPACE_____ \n")
# print("Action Space Shape", env.action_space.n)
# print("Action Space Sample", env.action_space.sample()) # Take a random action

In [4]:
# Based off Hugging Face policy gradient code
# Changes by CS 175 project group:
#   - class inherits nn.Sequential rather than nn.Module
#   - change to constructor method and deletion of explicitly defined forward method
class Policy(nn.Sequential):
  def __init__(self, n_layers, h_size, dropout, s_size, a_size):
    layers = []

    in_features = s_size
    for i in range(n_layers):
      layers.append(nn.Linear(in_features, h_size))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(0))
      in_features = h_size
    layers.append(nn.Linear(in_features, a_size))
    layers.append(nn.LogSoftmax(dim=1))

    super().__init__(*layers)

  def act(self, state):
    state = torch.from_numpy(state).float().unsqueeze(0).to(device)
    probs = self.forward(state).cpu()
    m = Categorical(probs)
    action = m.sample()
    return action.item(), m.log_prob(action)

In [17]:
# Based off Hugging Face policy gradient code
# Changes by CS 175 project group:
#   - changes to reward for training
#   - ensure changes to reward doesn't affect score output
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every, step_reward, ghost_reward, policy_file_name, step_reward_multiplier):
    scores_deque = deque(maxlen=100)
    scores = []
    for i_episode in tqdm(range(1, n_training_episodes+1)):
        saved_log_probs = []
        rewards = []
        state,game_env = env.reset()

        step_num = 0
        score_adjustments = 0
        cur_step_reward = step_reward

        for t in range(max_t):
            old_game_env = game_env

            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, done, _, game_env = env.step(action)
            
            # Equal reward for eating ghost
            if reward // 100 == 2:
              reward = reward - 200 + ghost_reward
              score_adjustments += 200 - ghost_reward
            elif reward // 100 == 4:
              reward = reward - 400 + ghost_reward
              score_adjustments += 400 - ghost_reward
            elif reward // 100 == 8:
              reward = reward - 800 + ghost_reward
              score_adjustments += 800 - ghost_reward
            elif reward // 100 == 16:
              reward = reward - 1600 + ghost_reward
              score_adjustments += 1600 - ghost_reward
                
            if reward % 100 == 10:
                cur_step_reward = step_reward
            else:
                cur_step_reward *= step_reward_multiplier
                
            reward += step_reward
            score_adjustments -= step_reward
            
            rewards.append(reward)

            if done:
                break

        reward_sum = sum(rewards) + score_adjustments
        scores_deque.append(reward_sum)
        scores.append(reward_sum)

        returns = deque(maxlen=max_t)
        n_steps = len(rewards)

        # Compute the discounted returns at each timestep,
        # as the sum of the gamma-discounted return at time t (G_t) + the reward at time t

        ## We compute this starting from the last timestep to the first, to avoid redundant computations

        ## appendleft() function of queues appends to the position 0
        ## We use deque instead of lists to reduce the time complexity

        for t in range(n_steps)[::-1]:
          disc_return_t = rewards[t] + gamma * (disc_return_t if t + 1 < n_steps else 0)
          returns.appendleft(disc_return_t)

        ## standardization for training stability
        eps = np.finfo(np.float32).eps.item()

        ## eps is added to the standard deviation of the returns to avoid numerical instabilities
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + eps)

        policy_loss = []
        for log_prob, disc_return in zip(saved_log_probs, returns):
            policy_loss.append(-log_prob * disc_return)
        policy_loss = torch.cat(policy_loss).sum()

        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()

        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))

    torch.save(policy, policy_file_name)
    return scores

In [20]:
hyperparameters = {
    "gamma": 1e-2,
    "n_layers": 2,
    "h_size": 274,
    "dropout": 0.4217,
    "lr": 0.002333,
    "step_reward": -1,
    "step_reward_multiplier": 1.01,
    "ghost_reward": -72,
    "optimizer": "Adam",
    "policy_file_name": "MsPacManPG_optimized.pt",
    "n_training_episodes": 100,
    "n_evaluation_episodes": 10,
    "max_t": 5000,
    "env_id": "ALE/MsPacman-ram-v5",
    "s_size": 128,
    "a_size": 5,
}

In [21]:
torch.manual_seed(50)
# Create policy and place it to the device
policy = Policy(hyperparameters["n_layers"], hyperparameters["h_size"],
                hyperparameters["dropout"], hyperparameters["s_size"],
                hyperparameters["a_size"]).to(device)
optimizer = getattr(optim, hyperparameters["optimizer"])(policy.parameters(), lr=hyperparameters["lr"])

In [22]:
# Train and save neural network
scores = reinforce(policy,
                   optimizer,
                   hyperparameters["n_training_episodes"],
                   hyperparameters["max_t"],
                   1 - hyperparameters["gamma"],
                   10, hyperparameters["step_reward"],
                   hyperparameters["ghost_reward"],
                   hyperparameters["policy_file_name"], 
                   hyperparameters["step_reward_multiplier"])

 10%|████████                                                                         | 10/100 [00:09<01:24,  1.06it/s]

Episode 10	Average Score: 266.00


 20%|████████████████▏                                                                | 20/100 [00:18<01:21,  1.02s/it]

Episode 20	Average Score: 357.50


 30%|████████████████████████▎                                                        | 30/100 [00:28<01:03,  1.10it/s]

...

Episode 80	Average Score: 447.50


 90%|████████████████████████████████████████████████████████████████████████▉        | 90/100 [01:38<00:11,  1.12s/it]

Episode 90	Average Score: 449.00


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:49<00:00,  1.09s/it]

Episode 100	Average Score: 450.50


In [23]:
def evaluate_agent(env, max_steps, n_eval_episodes, policy):
    """
    Evaluate the agent for ``n_eval_episodes`` episodes and returns average reward and std of reward.
    :param env: The evaluation environment
    :param n_eval_episodes: Number of episode to evaluate the agent
    :param policy: The Reinforce agent
    """
    episode_rewards = []
    for episode in range(n_eval_episodes):
        state, game_env = env.reset()
        step = 0
        done = False
        total_rewards_ep = 0

        for step in range(max_steps):
            action, _ = policy.act(state)
            new_state, reward, done, _, game_env = env.step(action)
            total_rewards_ep += reward

            if done:
                break
            state = new_state
        episode_rewards.append(total_rewards_ep)
    mean_reward = np.mean(episode_rewards)
    std_reward = np.std(episode_rewards)

    return mean_reward, std_reward

In [ ]:
# # Load saved policy
# policy = torch.load(hyperparameters["policy_file_name"])
# policy.eval()

In [24]:
mean, std = evaluate_agent(env, hyperparameters["max_t"], 10, policy)
print(f"Mean: {mean}; Std: {std}")

Mean: 506.0; Std: 266.39069052802876


In [25]:
def record_video(env, policy, out_directory, fps=30):
  """
  Generate a replay video of the agent
  :param env
  :param Qtable: Qtable of our agent
  :param out_directory
  :param fps: how many frame per seconds (with taxi-v3 and frozenlake-v1 we use 1)
  """
  done = False
  state, _ = env.reset()
  score = 0
  while not done:
    # Take the action (index) that have the maximum expected future reward given that state
    action, _ = policy.act(state)
    new_state, reward, done, _, game_env = env.step(action)
    score += reward
  print(score)
  imageio.mimsave(out_directory, env.render(), fps=fps)

In [26]:
replay_env = gym.make("ALE/MsPacman-ram-v5", render_mode="rgb_array_list")
record_video(replay_env, policy, './MsPacMan_replay.mp4')

480.0
